In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import imageio
import os


In [33]:
# Load processed data
df = pd.read_csv("../data/processed/labeled_posts.csv")
df['created_at'] = pd.to_datetime(df['created_at'], format='mixed', utc=True)
df['date_bin'] = df['created_at'].dt.to_period("W").astype(str)  # Weekly aggregation

# Funnel stages & colors
funnel_stages = ['Awareness', 'Interest', 'Trust', 'Advocacy', 'Drop-Off']
colors = {
    'Awareness': '#1f77b4',
    'Interest': '#ff7f0e',
    'Trust': '#2ca02c',
    'Advocacy': '#9467bd',
    'Drop-Off': '#d62728'
}

# Aggregate stage counts per week
grouped = df.groupby(['date_bin', 'funnel_stage']).size().unstack()
grouped = grouped.reindex(columns=colors.keys(), fill_value=0)
frames_dir = "../assets/frames"
os.makedirs(frames_dir, exist_ok=True)

# Create chart frames
ymax = grouped.sum().max()  # keep y-axis consistent
filenames = []
for i, date in enumerate(grouped.index):
    counts = grouped.iloc[:i+1].sum()
    plt.figure(figsize=(10, 6))
    plt.bar(funnel_stages, counts[funnel_stages], color=[colors[s] for s in funnel_stages])
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.gca().spines['right'].set_visible(False)
    plt.gca().spines['top'].set_visible(False)
    plt.title(f"Sentiment Funnel Progression – Week Ending {date}", fontsize=14)
    plt.xlabel("Funnel Stage")
    plt.ylabel("Message Count")
    plt.ylim(top=ymax)
    plt.tight_layout()
    filepath = f"{frames_dir}/frame_{i}.png"
    plt.savefig(filepath)
    filenames.append(filepath)
    plt.close()

# Build GIF
with imageio.get_writer("../assets/funnel_gif.gif", mode='I', duration=0.5) as writer:
    for filename in filenames:
        image = imageio.imread(filename)
        writer.append_data(image)

print("Funnel GIF created at assets/funnel_gif.gif")

/var/folders/n0/7l1_05910y303wy7z45_m3kr0000gn/T/ipykernel_50093/154633929.py:4: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['date_bin'] = df['created_at'].dt.to_period("W").astype(str)  # Weekly aggregation
/var/folders/n0/7l1_05910y303wy7z45_m3kr0000gn/T/ipykernel_50093/154633929.py:45: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


Funnel GIF created at assets/funnel_gif.gif
